In [1]:
#### Jupyter Basic Setting ####
import pandas as pd,numpy as np, os,sys # np,pd,os,sys
import matplotlib.pyplot as plt,seaborn as sns  # 시각화
import warnings; warnings.filterwarnings('ignore')  # 경고 무시
# 다른 directory Module 불러오기위한 Setting 
parent_dir = os.path.dirname(os.getcwd() )
sys.path.append(parent_dir)
from Module.Functions import *
Service.plotSetting("seaborn-v0_8")
## Data dictionary save 
# np.save("../Data/DataDictionary/subway_dict_test.npy",subway_dict_22_23) ## 99.8Mb
# subway_dict_22_23 =np.load("../Data/DataDictionary/subway_dict_test.npy",allow_pickle=True).tolist()
# subway_dict_22_23.keys()


___## OS platform 한글 세팅완료 ## ___


In [2]:
StationInfo_dict = {}
print(parent_dir)
for i in os.listdir(parent_dir):
    # print(i)
    file_order = 0
    if i =='Data':
        print(f"!{i}")
        for idx, filename in enumerate(os.listdir(os.path.join(parent_dir,i))):
            # print(" |-",j)
            if 'StationInfo' in filename:
                file_order += 1
                print(f"{file_order}. -> ",filename)
                key_name=filename.split(".csv")[0]
                # print(key_name)
                StationInfo_dict[f'{key_name}'] = pd.read_csv(os.path.join(parent_dir,i,filename))
print(Service.colored_text("  >> 📌Data folder 에 있는 년도별 Station Info 를  StationInfo_dict 에 저장합니다.", 'blue'))
print("[[StationInfo keys]]")
for i in StationInfo_dict.keys():
    print("  |-",Service.colored_text(i,'cyan'))

data_name = 'StationInfo_23_0'
print(Service.colored_text(f"✅ {data_name} :역사정보 \n",'yellow'),StationInfo_dict[data_name].head(3))
latlng = pd.read_csv("../Data/seoul_subway_latlon_zenzen.csv")
latlng.rename( {'고유역번호(외부역코드)':'역사코드'}, inplace=True, axis=1)
latlng.drop(['역명','호선','환승역수'], axis=1,inplace=True)
print(Service.colored_text(f"✅ {data_name} : 역사별 위도경도 : \n",'yellow'),latlng.head(3))

## 위도 경도 데이터 merge 
StationInfo_dict[data_name] = pd.merge(StationInfo_dict[data_name],latlng, on='역사코드',how='inner' )
print(Service.colored_text('✅ Merged data check(tail)','yellow'))
print(StationInfo_dict[data_name].tail(3))

# 호선별 배차시간데이터 
savefilename_indexName = data_name.split('StationInfo')[-1]
line_배치_dict ={}
for i in range(1,9):
    line_배치_dict[f'{i}호선'] = pd.read_csv(f'../Data/지하철배차시간데이터/StationInfo{savefilename_indexName}_{i}_호선배차.csv')
    line_배치_dict[f'{i}호선'] = \
        Service.table_merge_subwayInfo_dispatch(
            StationInfo_dict[data_name],
            line_배치_dict[f'{i}호선'],
            histPlot=False
            )
# line_배치_dict['7호선'].tail()


/Users/forrestdpark/Desktop/PDG/Python_/Project/HaruSijack/DataAnalysis
!Data
1. ->  StationInfo_2023-10-31.csv
2. ->  StationInfo_2022-12-31.csv
3. ->  StationInfo_22.csv
4. ->  StationInfo_23_2.csv
5. ->  StationInfo_23_0.csv
6. ->  StationInfo_23_1.csv
7. ->  StationInfo.csv
  >> 📌Data folder 에 있는 년도별 Station Info 를  StationInfo_dict 에 저장합니다.
[[StationInfo keys]]
  |- StationInfo_2023-10-31
  |- StationInfo_2022-12-31
  |- StationInfo_22
  |- StationInfo_23_2
  |- StationInfo_23_0
  |- StationInfo_23_1
  |- StationInfo
✅ StationInfo_23_0 :역사정보 
    역사코드  역이름  호선  환승역수
0   150  서울역   1     2
1   426  서울역   4     2
2   151   시청   1     2
✅ StationInfo_23_0 : 역사별 위도경도 : 
    역사코드   latitude   longitude
0   150  37.554679  126.970607
1   426  37.554679  126.970607
2   151  37.565345  126.977198
✅ Merged data check(tail)
     역사코드  역이름  호선  환승역수   latitude   longitude
273  2828  남위례   8     1  37.462723  127.139351
274  2649   신내   6     1  37.612491  127.104271
275  2753  까치울   7     1 

In [3]:
# 전체학습데이터 생성 
# 기본 승하차데이터의 승하차 칼럼들을 다시 정제 해야함. 
mlTable_dict = {}
subway_dict_22_23 =np.load("../Data/DataDictionary/subway_dict_test.npy",allow_pickle=True).tolist()
for i,key in enumerate(subway_dict_22_23.keys()):
    print(Service.colored_text(f'🔹 {i} key : {key}','cyan'))
    mlTable_dict[key]= Service.data_preprocessing_toAnalysis(subway_dict_22_23,key)

print(Service.colored_text("########### 정제된 데이터에서 Feature Engineering 을 시작합니다 #################",'yellow'))

for i,key in enumerate(mlTable_dict.keys()):
    print(Service.colored_text(f'🔹 {i} key : {key}','cyan'))
    print(Service.colored_text(f">> 📌{key}data 의 수송일자 칼럼에서 요일(int)을 추출한뒤 4번째 칼럼으로 이동",'yellow'))
    mlTable_dict[key]= Service.dayToIntConvert(mlTable_dict[key], "수송일자")
    mlTable_dict[key] = Service.reorder_columns(mlTable_dict[key],col_name="요일",target_idx=4)
    
    ##날짜에서 년도,월, 주차
    mlTable_dict[key] = Service.date_Divid_Add_YMW_cols(mlTable_dict[key] ,'수송일자')
    ## 날짜에서 공휴일 데이터 추가
    mlTable_dict[key]= Service.holidaysToIntConvert(mlTable_dict[key],DateColName='수송일자')
    ## 수송일자 , 연번 삭제 시간관련데이터 앞으로 이동 
    for idx, col in enumerate(["년도","월","주차","공휴일"]):
        mlTable_dict[key] =Service.reorder_columns(col_name=col,df=mlTable_dict[key] ,target_idx=idx)
    mlTable_dict[key]['주중주말'] = ['SAT' if day in [5, 6] else 'DAY' for day in mlTable_dict[key]['요일']]
    mlTable_dict[key]=Service.reorder_columns(col_name='주중주말',df= mlTable_dict[key],target_idx=4)
# 시간대 칼럼 이름 변경 ~시 인원으로 
print(Service.colored_text("시간대 칼럼 이름 변경 ~시 인원으로",'yellow'))
for key in  mlTable_dict.keys():
    colnamelist =mlTable_dict[key].columns.tolist()
    시간대시작index=Service.indexFind(colnamelist,'06')[0]
    # print(*colnamelist[시간대시작index:])
    for colname in colnamelist[시간대시작index:]:
        # print(f'{colname[:2]}시인원')
        if '06시이전' == colname:
            mlTable_dict[key].rename({colname:f'05시인원'}, inplace=True, axis=1)    
        else:
            mlTable_dict[key].rename({colname:f'{colname[:2]}시인원'}, inplace=True, axis=1)



print(Service.colored_text("########### Feature Engineering 완료 #################",'yellow'))

🔹 0 key : subway23_0
columns ---👇
['연번', '수송일자', '호선', '역번호', '역명', '승하차구분', '06시이전', '06-07시간대', '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대', '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대', '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대', '24시이후']
 1. 연번을 삭제합니다. 
 2."역명" ->"역이름", "역번호"->"역사코드 ".
 3. 호선 데이터가 object 입니다. 
   - 1호선
   - 2호선
   - 3호선
 ..
 😀호선을 integer 로 만듭니다.
 3-1. 호선 을 제거한 이름 unique : , 1, 2, 3, 4, 5, 6, 7, 8
 ✅변경된 호선 칼럼의 data type : int64
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 <<< dataInfoProcessing. >>>
  1️⃣ Data row/colum numbers : 98603/25
  2️⃣ null ceck 결과Null 없는 clean data!
  3️⃣ Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[수송일

In [4]:
# # 시간대 칼럼 이름 변경 
# #index findeing
# def indexFind(colnamelist, search_target_word):
#     # 해당 단어가 존재하는 칼럼의 이름이있는 칼럼의 indx를 출력합니다. 
#     indices = np.where([search_target_word in col for col in colnamelist])[0]
#     return indices

# colnamelist = mlTable_dict['subway23_0'].columns.tolist()
# 시간대시작index =indexFind(colnamelist,'06')[0]
# print(시간대시작index)

# # Service.reorder_columns(target_idx=,col_name='주중주말',df=mlTable_dict['subway23_0'])

# # 
# for key in  mlTable_dict.keys():
#     colnamelist =mlTable_dict[key].columns.tolist()
#     시간대시작index=Service.indexFind(colnamelist,'06')[0]
#     # print(*colnamelist[시간대시작index:])
#     for colname in colnamelist[시간대시작index:]:
#         # print(f'{colname[:2]}시인원')
#         if '06시이전' == colname:
#             mlTable_dict[key].rename({colname:f'05시인원'}, inplace=True, axis=1)    
#         else:
#             mlTable_dict[key].rename({colname:f'{colname[:2]}시인원'}, inplace=True, axis=1)

mlTable_dict['subway23_0'].columns


Index(['년도', '월', '주차', '공휴일', '주중주말', '수송일자', '호선', '역사코드', '역이름', '요일',
       '승하차구분', '05시인원', '06시인원', '07시인원', '08시인원', '09시인원', '10시인원', '11시인원',
       '12시인원', '13시인원', '14시인원', '15시인원', '16시인원', '17시인원', '18시인원', '19시인원',
       '20시인원', '21시인원', '22시인원', '23시인원', '24시인원'],
      dtype='object')

In [5]:
for i in line_배치_dict.keys():
    print(line_배치_dict[i].columns)

Index(['역사코드', '역이름', '호선', '환승역수', '주말', '주중', 'latitude', 'longitude',
       '주중주말', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24'],
      dtype='object')
Index(['역사코드', '역이름', '호선', '환승역수', '주말', '주중', 'latitude', 'longitude',
       '주중주말', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', 'END'],
      dtype='object')
Index(['역사코드', '역이름', '호선', '환승역수', '주말', '주중', 'latitude', 'longitude',
       '주중주말', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24'],
      dtype='object')
Index(['역사코드', '역이름', '호선', '환승역수', '주말', '주중', 'latitude', 'longitude',
       '주중주말', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24'],
      dtype='object')
Index(['역사코드', '역이름', '호선', '환승역수', '주말',

In [6]:
# 배차시간표는 Line 별로되어있기때문에 나눠서 머신러닝 만듬... 

for key in mlTable_dict.keys():
    mlTable=mlTable_dict[key]
    
    mlTable_line_dict  ={}
    for line in range(1,9):
        print(mlTable[mlTable['호선']==line])
        try: 
            mlt= mlTable[mlTable['호선']==line]
            print(mlt.columns)
            mlTable_line_dict[f'{key}_{line}호선']= pd.merge(line_배치_dict[f'{line}호선'],mlt,on = ['역사코드','주중주말','호선','역이름'])
        except: pass
    mlTable_dict[key]=mlTable_line_dict
mlTable_dict['subway23_0']

         년도  월  주차  공휴일 주중주말        수송일자  호선  역사코드  역이름  요일  ... 15시인원  16시인원  \
0      2022  1  52    0  DAY  2023-01-01   1   150  서울역   0  ...  2655   2509   
1      2022  1  52    0  DAY  2023-01-01   1   150  서울역   0  ...  2282   2295   
2      2022  1  52    0  DAY  2023-01-01   1   151   시청   0  ...   843    895   
3      2022  1  52    0  DAY  2023-01-01   1   151   시청   0  ...   602    575   
4      2022  1  52    0  DAY  2023-01-01   1   152   종각   0  ...  1145   1402   
...     ... ..  ..  ...  ...         ...  ..   ...  ...  ..  ...   ...    ...   
98072  2023  6  26    1  SAT  2023-06-30   1   157  제기동   5  ...  1446   1162   
98073  2023  6  26    1  SAT  2023-06-30   1   158  청량리   5  ...  1665   1847   
98074  2023  6  26    1  SAT  2023-06-30   1   158  청량리   5  ...  1584   1456   
98075  2023  6  26    1  SAT  2023-06-30   1   159  동묘앞   5  ...   833    965   
98076  2023  6  26    1  SAT  2023-06-30   1   159  동묘앞   5  ...   723    683   

       17시인원  18시인원  19시인원 

{'subway23_0_1호선':       역사코드  역이름  호선  환승역수     주말     주중   latitude   longitude 주중주말   05  ...  \
 0      150  서울역   1     2  False   True  37.554679  126.970607  DAY  5.5  ...   
 1      150  서울역   1     2  False   True  37.554679  126.970607  DAY  5.5  ...   
 2      150  서울역   1     2  False   True  37.554679  126.970607  DAY  5.5  ...   
 3      150  서울역   1     2  False   True  37.554679  126.970607  DAY  5.5  ...   
 4      150  서울역   1     2  False   True  37.554679  126.970607  DAY  5.5  ...   
 ...    ...  ...  ..   ...    ...    ...        ...         ...  ...  ...  ...   
 3615   159  동묘앞   1     2   True  False  37.573217  127.016364  SAT  4.5  ...   
 3616   159  동묘앞   1     2   True  False  37.573217  127.016364  SAT  4.5  ...   
 3617   159  동묘앞   1     2   True  False  37.573217  127.016364  SAT  4.5  ...   
 3618   159  동묘앞   1     2   True  False  37.573217  127.016364  SAT  4.5  ...   
 3619   159  동묘앞   1     2   True  False  37.573217  127.016364  SAT  4.5  ...  

In [7]:

## 배차 시간대 칼럼 이름 ~배차로 바꾸기 

for key in mlTable_dict.keys():
    for line in mlTable_dict[key].keys():
        table = mlTable_dict[key][line]
        colnamelist = table.columns
        배차StartIndex = Service.indexFind(colnamelist=colnamelist.tolist(),search_target_word='05')[0]
        배차FinalIndex = Service.indexFind(colnamelist=colnamelist.tolist(),search_target_word='24')[0]
        for colname in colnamelist[배차StartIndex:배차FinalIndex+1]:
            # print(f'{colname[:2]}시인원')
            table.rename({colname:f'{colname[:2]}배차'}, inplace=True, axis=1)
        mlTable_dict[key][line]=table

mlTable_dict.keys()
# mlTable_dict['subway23_0']['subway23_0_1'].columns



dict_keys(['subway23_0', 'subway23_1', 'subway23_2', 'subway22'])

In [8]:
# 칼럼 순서이동 및 쓸데없는 칼럼 버리기 
move_target_colname_list =[ '년도', '월', '주차', '공휴일', '수송일자', '요일', '승하차구분']

# for line in mlTable_dict['subway23_0']['subway23_0_1']:
    
for key in mlTable_dict.keys():
    for line in mlTable_dict[key].keys():
        table = mlTable_dict[key][line]
        table.drop('주중주말', axis=1, inplace=True)
        table.drop('수송일자', axis=1, inplace=True)
        table.drop('년도', axis=1, inplace=True)
        mlTable_dict[key][line]=table

In [9]:
mlTable_dict['subway23_0']['subway23_0_1호선']['승하차구분']#.columns#.iloc[:3,:23].head()

0       승차
1       하차
2       승차
3       하차
4       승차
        ..
3615    하차
3616    승차
3617    하차
3618    승차
3619    하차
Name: 승하차구분, Length: 3620, dtype: object

In [10]:
mlTable_dict['subway23_0']['subway23_0_1호선'].columns

Index(['역사코드', '역이름', '호선', '환승역수', '주말', '주중', 'latitude', 'longitude',
       '05배차', '06배차', '07배차', '08배차', '09배차', '10배차', '11배차', '12배차', '13배차',
       '14배차', '15배차', '16배차', '17배차', '18배차', '19배차', '20배차', '21배차', '22배차',
       '23배차', '24배차', '월', '주차', '공휴일', '요일', '승하차구분', '05시인원', '06시인원',
       '07시인원', '08시인원', '09시인원', '10시인원', '11시인원', '12시인원', '13시인원', '14시인원',
       '15시인원', '16시인원', '17시인원', '18시인원', '19시인원', '20시인원', '21시인원', '22시인원',
       '23시인원', '24시인원'],
      dtype='object')

In [12]:
test = mlTable_dict['subway23_0']['subway23_0_1호선'].columns

def MLPredciton_ver02(featureTable,saveFileName) :
        """
        ##### 📌 Description : train,target이 모두있는 feature table 을 입력받아 ML model 별 score 출력후 모델 반환
        ##### 📌 Date : 2024.06.13
        ##### 📌 Author : pdg
        ##### 📌 Detail: 승하차구분없고, in,out 구분이 없다. 
            🔸 Returns: - 
        ##### 📌 Updata:
            🟦 2024.06.13 by pdg : 모델 함수 만듬. 
        """
        import pandas as pd, numpy as np
        import matplotlib.pyplot as plt 
        from sklearn.model_selection import train_test_split
        from sklearn.multioutput import MultiOutputRegressor
        from sklearn.neighbors import KNeighborsRegressor

        input_column=['월', '주차', '공휴일', '요일', '역사코드', '주중', '주말',
            'latitude', 'longitude', '05배차', '06배차', '07배차', '08배차', '09배차', '10배차',
            '11배차', '12배차', '13배차', '14배차', '15배차', '16배차', '17배차', '18배차', '19배차',
            '20배차', '21배차', '22배차', '23배차', '24배차']
        target_column=['05시인원', '06시인원', '07시인원',
        '08시인원', '09시인원', '10시인원', '11시인원', '12시인원', '13시인원', '14시인원', '15시인원',
        '16시인원', '17시인원', '18시인원', '19시인원', '20시인원', '21시인원', '22시인원', '23시인원',
        '24시인원']
        
        mlTable_승차 = featureTable[featureTable['승하차구분']=='승차']
        mlTable_하차 = featureTable[featureTable['승하차구분']=='하차']
        mlTable_승차_input = mlTable_승차[input_column]
        mlTable_승차_target =mlTable_승차[target_column]
        mlTable_하차_input= mlTable_하차[input_column]
        mlTable_하차_target= mlTable_하차[target_column]
        # mlTable_dict ={
        # 'mlTable_승차':mlTable_승차,
        # 'mlTable_하차':mlTable_하차,
        # "mlTable_승차_input":mlTable_승차_input,
        # "mlTable_승차_target":mlTable_승차_target,
        # "mlTable_하차_input":mlTable_하차_input,
        # "mlTable_하차_target":mlTable_하차_target,
        # }
        
        train_input_승차, test_input_승차, train_target_승차, test_target_승차 = \
            train_test_split(mlTable_승차_input,
                            mlTable_승차_target, 
                            test_size=0.2,
                            # random_state=42
                            )
        train_input_하차, test_input_하차, train_target_하차, test_target_하차 = \
            train_test_split(mlTable_하차_input,
                            mlTable_하차_target, 
                            test_size=0.2,
                            # random_state=42
                            )
        ml_train_test_dict ={
            'train_input_승차':train_input_승차,
            'test_input_승차':test_input_승차,
            'train_target_승차':train_target_승차,
            'test_target_승차':test_target_승차,
            'train_input_하차':train_input_하차,
            'test_input_하차':test_input_하차,
            'train_target_하차':train_target_하차,
            'test_target_하차':test_target_하차
        }
        np.save(f'../MLModels/TrainingTargetDataSet_dictionary/{saveFileName}.npy',ml_train_test_dict)
        # return ml_train_test_dict

for key in mlTable_dict.keys():
    for line in mlTable_dict[key].keys():
        featureTable=mlTable_dict[key][line]
        MLPredciton_ver02(featureTable=featureTable, saveFileName=f'{line}_data_dict')


KeyError: '승하차구분'

In [23]:
data_dict=np.load('../MLModels/TrainingTargetDataSet_dictionary/subway23_0_1호선_data_dict.npy',allow_pickle=True).tolist()
def regression_model(data_dict,승하차,saveFileName):
    import pandas as pd, numpy as np
    import matplotlib.pyplot as plt 
    from sklearn.model_selection import train_test_split
    from sklearn.multioutput import MultiOutputRegressor
    from sklearn.neighbors import KNeighborsRegressor
    knn_regressor = KNeighborsRegressor(n_neighbors=3)
    ## Multi Output Setting
    multi_output_regressor = MultiOutputRegressor(knn_regressor)
    multi_output_regressor.fit(data_dict[f'train_input_{승하차}'], data_dict[f'train_target_{승하차}'])
    score = multi_output_regressor.score(data_dict[f'test_input_{승하차}'], data_dict[f'test_target_{승하차}'])
    predictions= multi_output_regressor.predict(data_dict[f'test_input_{승하차}'])
    print(Service.colored_text(f'{saveFileName}_{승하차}예측--','yellow'))
    print(Service.colored_text(f'✅Model score({승하차}): {score}','red'))
    print("주차     요일 시간대별 예측 :",*[f"{i}시" for i in range(5,25)], sep='\t')
    for idx,시간대별예측 in enumerate(predictions):
        if idx <3:
            주차 = data_dict[f'test_input_{승하차}'].to_numpy()[idx][1]
            요일 = data_dict[f'test_input_{승하차}'].to_numpy()[idx][3]
            실제치 = data_dict[f'test_target_{승하차}'].to_numpy()[idx]
            match 요일:
                case 요일 if 요일 == 0: 요일_str = '일'; 
                case 요일 if 요일 == 1: 요일_str = '월'; 
                case 요일 if 요일 == 2: 요일_str = '화'; 
                case 요일 if 요일 == 3: 요일_str = '수'; 
                case 요일 if 요일 == 4: 요일_str = '목'; 
                case 요일 if 요일 == 5: 요일_str = '금'; 
                case 요일 if 요일 == 6: 요일_str = '토'; 
                case _:print()
            print(f"{주차}주차 {요일_str}요일 시간대별 예측 :", *list(map(int,(시간대별예측))), sep='\t')
            print(f"{주차}주차 {요일_str}요일 시간대별 실제 :", *실제치, sep='\t')
            print("---"*200)
    import joblib ## model 저장 용 함수 
    filename = f'../MLModels/knn_regressor_{saveFileName}_{승하차}.h5'
    print(f"😀😀😀😀{filename} 을 저장합니다 😀😀😀")
    joblib.dump(multi_output_regressor, filename)
        #return multi_output_regressor
# regression_model(data_dict,'승차', saveFileName=)
# regression_model(data_dict,'하차')


In [24]:
for i in os.listdir('../MLModels/TrainingTargetDataSet_dictionary'):
    print(i)
    data_dict=np.load(os.path.join('../MLModels/TrainingTargetDataSet_dictionary/',i),allow_pickle=True).tolist()
    regression_model(data_dict,'승차', saveFileName=i.split('_data_dict.npy')[0])
    regression_model(data_dict,'하차', saveFileName=i.split('_data_dict.npy')[0])
    

subway23_0_8호선_data_dict.npy
subway23_0_8호선_승차예측--
✅Model score(승차): 0.8192247184144341
주차     요일 시간대별 예측 :	5시	6시	7시	8시	9시	10시	11시	12시	13시	14시	15시	16시	17시	18시	19시	20시	21시	22시	23시	24시
2주차 목요일 시간대별 예측 :	412	831	2091	2729	1271	689	580	609	582	508	474	535	748	862	354	233	180	118	52	13
2주차 목요일 시간대별 실제 :	416	850	2161	2706	1279	688	611	572	597	538	466	567	768	891	350	222	144	137	49	18
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
24주차 토요일 시간대별 예측 :

23년은 여러개로쪼개져있으니 예측률이 다양해도 맞추는게 의미가 없다.
빨리 22,이랑 다 합친걸로 하는걸 하도록 하자. 